In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kaitangata,-46.2817,169.8464,77.67,42,100,16.78,NZ,1611621764
1,1,pyapon,16.2833,95.6833,67.05,57,0,3.69,MM,1611621765
2,2,leningradskiy,69.3833,178.4167,-15.14,86,100,18.03,RU,1611621765
3,3,ammon,43.4696,-111.9666,17.60,73,20,4.61,US,1611621765
4,4,barrow,71.2906,-156.7887,-5.80,84,1,12.66,US,1611621602


In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,rocha,-34.4833,-54.3333,71.56,86,0,7.72,UY,1611621766
141,141,opunake,-39.4556,173.8583,75.99,76,0,3.00,NZ,1611621785
143,143,saint-joseph,-21.3667,55.6167,77.00,73,0,6.91,RE,1611621785
168,168,chuy,-33.6971,-53.4616,73.58,85,0,9.40,UY,1611621789
235,235,salalah,17.0151,54.0924,71.60,35,0,9.22,OM,1611621798
306,306,perth,-31.9333,115.8333,73.99,52,0,4.61,AU,1611621712
334,334,nautla,20.2167,-96.7833,73.15,89,0,7.47,MX,1611621814
347,347,saint-philippe,-21.3585,55.7679,77.00,73,0,6.91,RE,1611621816
414,414,kayes,14.0000,-11.0000,70.97,17,0,3.96,ML,1611621825
468,468,harper,4.3750,-7.7169,77.72,86,0,7.02,LR,1611621834


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
14,rocha,UY,-34.4833,-54.3333,
141,opunake,NZ,-39.4556,173.8583,
143,saint-joseph,RE,-21.3667,55.6167,
168,chuy,UY,-33.6971,-53.4616,
235,salalah,OM,17.0151,54.0924,
306,perth,AU,-31.9333,115.8333,
334,nautla,MX,20.2167,-96.7833,
347,saint-philippe,RE,-21.3585,55.7679,
414,kayes,ML,14.0000,-11.0000,
468,harper,LR,4.3750,-7.7169,


In [6]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
14,rocha,UY,-34.4833,-54.3333,
141,opunake,NZ,-39.4556,173.8583,
143,saint-joseph,RE,-21.3667,55.6167,
168,chuy,UY,-33.6971,-53.4616,
235,salalah,OM,17.0151,54.0924,
306,perth,AU,-31.9333,115.8333,
334,nautla,MX,20.2167,-96.7833,
347,saint-philippe,RE,-21.3585,55.7679,
414,kayes,ML,14.0000,-11.0000,
468,harper,LR,4.3750,-7.7169,


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig